# Tensorflow Recommenders Tutorial
https://www.tensorflow.org/recommenders/examples

## Imports

In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
import tensorflow_recommenders as tfrs

## Preparing the Dataset

In [3]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\movielens\100k-movies\0.1.1.incomplete9FRSMR\movielens-train.tfrecord*...:   0…

Dataset movielens downloaded and prepared to ~\tensorflow_datasets\movielens\100k-movies\0.1.1. Subsequent calls will reuse this data.


In [4]:
# Examine a record

## Note: the tensorflow dataset object has a unique API.  Slicing tensors to human-readable format can be tricky

for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7], dtype=int64),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [6]:
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_genres': array([4], dtype=int64),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [7]:
# Feature Reduction

## We'll reduce features for the first model

ratings = ratings.map(
    lambda x: {
        "movie_title": x["movie_title"],
        "user_id": x["user_id"],
    }
)
movies = movies.map(lambda x: x["movie_title"])

In [9]:
# Split Data Set into Training and Evaluation Chunks

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [14]:
# Map categorical features to embeddings for models

movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

len(unique_movie_titles), unique_movie_titles[:10]

(1664,
 array([b"'Til There Was You (1997)", b'1-900 (1994)',
        b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
        b'2 Days in the Valley (1996)',
        b'20,000 Leagues Under the Sea (1954)',
        b'2001: A Space Odyssey (1968)',
        b'3 Ninjas: High Noon At Mega Mountain (1998)',
        b'39 Steps, The (1935)'], dtype=object))

# Implementing A Model

Two-Tower Retrieval Model

### The Query Tower

In [15]:
embedding_dimension = 32

In [18]:
# Preprocessing Layer to create user embeddings

user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
    # Add an additional embedding to account for unknown tokens
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

### The Candidate Tower

In [20]:
# Creating a similarly build embedding for movies

movie_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
    tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [22]:
movie_model

## Metrics

Accuracy can be a trick to define in nearest neighbor models.  Here, we'll check the relative distance between positive pairs and all other candidates and hope the difference is high/relevant.

In [23]:
metrics = tfrs.metrics.FactorizedTopK(
    candidates=movies.batch(128).map(movie_model)
)

## Loss

We can use the metric (factorized top K) as a loss function

In [24]:
task = tfrs.tasks.Retrieval(
    metrics=metrics
)

## The Full Model

With the loss function and accuracy metric defined, data prepared, we can now define our full model

In [25]:
class MovieLensModel(tfrs.Model):
    
    def __init__(self, user_model, movie_model):
        super().__init__()
        self.movie_model: tf.keras.Model = movie_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task  # note task defined above
            
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Pick out user features and feed to user model for user embeddings
        user_embeddings = self.user_model(features["user_id"])
        # Pick out movie features to get movie embeddings
        positive_movie_embeddings = self.movie_model(features["movie_title"])
        
        # Calculate loss and metrics, return
        return self.task(user_embeddings, positive_movie_embeddings)
        

## Fit and Evaluate

In [26]:
model = MovieLensModel(user_model=user_model, movie_model=movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [30]:
# shuffle, batch, cache training/eval data
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [28]:
# Train model
model.fit(cached_train, epochs=3)

Epoch 1/3
10/10 [==============================] - 12s 943ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0098 - factorized_top_k/top_10_categorical_accuracy: 0.0206 - factorized_top_k/top_50_categorical_accuracy: 0.1005 - factorized_top_k/top_100_categorical_accuracy: 0.1775 - loss: 69885.1108 - regularization_loss: 0.0000e+00 - total_loss: 69885.1108
Epoch 2/3
10/10 [==============================] - 9s 948ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0027 - factorized_top_k/top_5_categorical_accuracy: 0.0190 - factorized_top_k/top_10_categorical_accuracy: 0.0381 - factorized_top_k/top_50_categorical_accuracy: 0.1697 - factorized_top_k/top_100_categorical_accuracy: 0.2930 - loss: 67523.3679 - regularization_loss: 0.0000e+00 - total_loss: 67523.3679
Epoch 3/3
10/10 [==============================] - 11s 1s/step - factorized_top_k/top_1_categorical_accuracy: 0.0036 - factorized_top_k/top_5_categorical_accuracy: 0

In [32]:
# Evaluate Model
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 2s 312ms/step - factorized_top_k/top_1_categorical_accuracy: 9.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0094 - factorized_top_k/top_10_categorical_accuracy: 0.0227 - factorized_top_k/top_50_categorical_accuracy: 0.1247 - factorized_top_k/top_100_categorical_accuracy: 0.2326 - loss: 31079.0641 - regularization_loss: 0.0000e+00 - total_loss: 31079.0641


{'factorized_top_k/top_1_categorical_accuracy': 0.0009500000160187483,
 'factorized_top_k/top_5_categorical_accuracy': 0.009399999864399433,
 'factorized_top_k/top_10_categorical_accuracy': 0.022749999538064003,
 'factorized_top_k/top_50_categorical_accuracy': 0.1246500015258789,
 'factorized_top_k/top_100_categorical_accuracy': 0.2326499968767166,
 'loss': 28244.7734375,
 'regularization_loss': 0,
 'total_loss': 28244.7734375}

## Making Predictions


In [37]:
# Create a model thiat takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset
index.index_from_dataset(
    tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Reccommendations for user 42: {titles[0, :3]})")

Reccommendations for user 42: [b'Bridges of Madison County, The (1995)'
 b'Father of the Bride Part II (1995)' b'Rudy (1993)'])


## Exporting the Model

In [43]:
# Export the query model
with tempfile.TemporaryDirectory() as tmp:
    path = os.path.join(tmp, "model")
    
    # save the index
    tf.saved_model.save(index, path)
    
    # load it back; can also be done via TensorFlow Serving
    loaded = tf.saved_model.load(path)
    
    # Pass a user id in, get top predicted movies back
    scores, titles = loaded(["3", "4", "42"])
    
    print(f"Reccommendations for user 42: {titles[:3, :3]})")
    

INFO:tensorflow:Assets written to: C:\Users\u1377381\AppData\Local\Temp\tmpbrfxyoiz\model\assets


INFO:tensorflow:Assets written to: C:\Users\u1377381\AppData\Local\Temp\tmpbrfxyoiz\model\assets


Reccommendations for user 42: [[b'Deep Rising (1998)' b'Truman Show, The (1998)' b'Mimic (1997)']
 [b'Mimic (1997)' b'Money Talks (1997)' b'Money Talks (1997)']
 [b'Bridges of Madison County, The (1995)'
  b'Father of the Bride Part II (1995)' b'Rudy (1993)']])


## Fine Tuning Candidates with Ranking

The candidate retrieval, indexing, step yielded a items in a region around the search query.  To order them by predicted preference, we'll also implement a ranking model.

In [44]:
# reload ratings data (can use same features for both, but check query input dim)

ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [46]:
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        # Compute embeddings for users.
        self.user_embeddings = tf.keras.Sequential([
          tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
          tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
        ])

        # Compute embeddings for movies.
        self.movie_embeddings = tf.keras.Sequential([
          tf.keras.layers.StringLookup(
            vocabulary=unique_movie_titles, mask_token=None),
          tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
        ])

        # Compute predictions.
        self.ratings = tf.keras.Sequential([
          # Learn multiple dense layers.
          tf.keras.layers.Dense(256, activation="relu"),
          tf.keras.layers.Dense(64, activation="relu"),
          # Make rating predictions in the final layer.
          tf.keras.layers.Dense(1)
        ])

    def call(self, inputs):

        user_id, movie_title = inputs

        user_embedding = self.user_embeddings(user_id)
        movie_embedding = self.movie_embeddings(movie_title)

        return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [47]:
RankingModel()((["42"], ["One Flew Over the Cuckoo's Nest (1975)"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.03740937]], dtype=float32)>

In [48]:
# New loss function for ranking model
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [49]:
# Rewrite MovieLens Model for ranking
class MovielensRankingModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
          loss = tf.keras.losses.MeanSquaredError(),
          metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
        return self.ranking_model(
            (features["user_id"], features["movie_title"]))

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        labels = features.pop("user_rating")

        rating_predictions = self(features)

        # The task computes the loss and the metrics.
        return self.task(labels=labels, predictions=rating_predictions)

In [51]:
model = MovielensRankingModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [53]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [54]:
model.fit(cached_train, epochs=5)

Epoch 1/5
10/10 [==============================] - 2s 37ms/step - root_mean_squared_error: 2.1718 - loss: 4.3303 - regularization_loss: 0.0000e+00 - total_loss: 4.3303
Epoch 2/5
10/10 [==============================] - 0s 28ms/step - root_mean_squared_error: 1.1227 - loss: 1.2602 - regularization_loss: 0.0000e+00 - total_loss: 1.2602
Epoch 3/5
10/10 [==============================] - 0s 26ms/step - root_mean_squared_error: 1.1162 - loss: 1.2456 - regularization_loss: 0.0000e+00 - total_loss: 1.2456
Epoch 4/5
10/10 [==============================] - 0s 24ms/step - root_mean_squared_error: 1.1073 - loss: 1.2252 - regularization_loss: 0.0000e+00 - total_loss: 1.2252
Epoch 5/5
10/10 [==============================] - 0s 22ms/step - root_mean_squared_error: 1.0935 - loss: 1.1942 - regularization_loss: 0.0000e+00 - total_loss: 1.1942


In [55]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 13ms/step - root_mean_squared_error: 1.0836 - loss: 1.1699 - regularization_loss: 0.0000e+00 - total_loss: 1.1699


{'root_mean_squared_error': 1.0835518836975098,
 'loss': 1.1517865657806396,
 'regularization_loss': 0,
 'total_loss': 1.1517865657806396}

In [56]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
    test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array([movie_title])
    })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
    print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.6522346]]
Dances with Wolves (1990): [[3.5988433]]
Speed (1994): [[3.5449886]]
